In [1]:
# Core
import os, json, numpy as np, pandas as pd, joblib, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

# Viz & metrics
import matplotlib.pyplot as plt

# Our project modules
import sys
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.features import NUM_COLS, CAT_COLS, make_preprocessor   # for column lists
from src.data_utils import load_and_clean, time_split, get_blocks
from src.profit_threshold import break_even_p, decisions_from_probs
from src.rl_cql import build_mdp_dataset, train_cql, policy_actions
# FQE lives under d3rlpy.ope; some versions expose DiscreteFQE separately
try:
    from d3rlpy.ope import DiscreteFQE as FQE, FQEConfig
except Exception:
    # fallback for versions that only expose FQE (works for both spaces, but we use it as discrete)
    from d3rlpy.ope import FQE, FQEConfig


# Paths
DATA_PATH   = Path("/Users/macaryan/Coding/Shodh_AI/lendingclub-policy-optimization/data/accepted_2007_to_2018Q4.csv")  # adjust if different
PREPROC_PKL = Path("/Users/macaryan/Coding/Shodh_AI/lendingclub-policy-optimization/notebooks/preprocessor.joblib")
ARTIFACTS   = Path(".")
ARTIFACTS.mkdir(exist_ok=True)

plt.rcParams["figure.figsize"] = (6,4)
pd.set_option("display.max_columns", 200)


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
META_COLS = ["issue_d","loan_amnt","int_rate"]

df = load_and_clean(DATA_PATH, NUM_COLS, CAT_COLS, META_COLS)

train, valid, test = time_split(df, train_end="2016-12-31", val_year="2017")

Xtr_raw, ytr, tr_meta = get_blocks(train, NUM_COLS, CAT_COLS)
Xva_raw, yva, va_meta = get_blocks(valid, NUM_COLS, CAT_COLS)
Xte_raw, yte, te_meta = get_blocks(test,  NUM_COLS, CAT_COLS)

print("Shapes raw:", Xtr_raw.shape, Xva_raw.shape, Xte_raw.shape)
print("Default rate (train/val/test):", round(ytr.mean(),4), round(yva.mean(),4), round(yte.mean(),4))


Shapes raw: (1129956, 20) (178817, 20) (65142, 20)
Default rate (train/val/test): 0.2025 0.2721 0.2717


In [4]:
# Save once
np.save("Xtr.npy", Xtr_raw); np.save("Xva.npy", Xva_raw); np.save("Xte.npy", Xte_raw)
np.save("ytr.npy", ytr); np.save("yva.npy", yva); np.save("yte.npy", yte)
tr_meta.to_parquet("tr_meta.parquet"); va_meta.to_parquet("va_meta.parquet"); te_meta.to_parquet("te_meta.parquet")


In [ ]:
# Reload instantly (no CSV parse)
Xtr = np.load("Xtr.npy"); Xva = np.load("Xva.npy"); Xte = np.load("Xte.npy")
ytr = np.load("ytr.npy"); yva = np.load("yva.npy"); yte = np.load("yte.npy")
import pandas as pd
tr_meta = pd.read_parquet("tr_meta.parquet")
va_meta = pd.read_parquet("va_meta.parquet")
te_meta = pd.read_parquet("te_meta.parquet")


In [5]:
preprocessor = joblib.load(PREPROC_PKL)

def transform_df(pre, X_df):
    X = pre.transform(X_df)
    if hasattr(X, "toarray"):
        X = X.toarray()
    return X

Xtr = transform_df(preprocessor, Xtr_raw)
Xva = transform_df(preprocessor, Xva_raw)
Xte = transform_df(preprocessor, Xte_raw)

Xtr.shape, Xva.shape, Xte.shape

((1129956, 63), (178817, 63), (65142, 63))

In [6]:
pte_path = ARTIFACTS / "test_predictions.npy"
if pte_path.exists():
    p_te = np.load(pte_path)                 # p(default | x)
    print("Loaded DL test probabilities:", p_te.shape)
else:
    p_te = None
    print("No test_predictions.npy found; skipping DL policy comparison.")

# Profit-aware DL policy (approve if p < p*)
if p_te is not None:
    thr_te = break_even_p(te_meta["int_rate"].values)
    dl_actions_te = (p_te < thr_te).astype(int)  # 1=approve, 0=deny


Loaded DL test probabilities: (65142,)


In [7]:
def realized_profit(actions, y_true, loan_amnt, int_rate):
    actions = np.asarray(actions).astype(int)
    y_true  = np.asarray(y_true).astype(int)
    loan_amnt = np.asarray(loan_amnt).astype(float)
    int_rate  = np.asarray(int_rate).astype(float)
    # reward when approve:
    approve_profit = np.where(y_true==0, loan_amnt*int_rate, -loan_amnt)
    # reward when deny:
    deny_profit = np.zeros_like(approve_profit)
    return np.where(actions==1, approve_profit, deny_profit)

def policy_summary(name, actions, y_true, loan_amnt, int_rate):
    r = realized_profit(actions, y_true, loan_amnt, int_rate)
    return {
        "policy": name,
        "n": int(len(r)),
        "approve_rate": float(np.mean(actions)),
        "avg_profit": float(np.mean(r)),
        "median_profit": float(np.median(r)),
        "p95_profit": float(np.percentile(r,95)),
        "p5_profit": float(np.percentile(r,5)),
        "total_profit": float(np.sum(r))
    }


In [8]:
from src.rl_cql import build_mdp_dataset, train_cql, policy_actions, fqe_estimate

# Build datasets
mdp_train = build_mdp_dataset(Xtr, tr_meta["loan_amnt"].values, tr_meta["int_rate"].values, ytr)
mdp_valid = build_mdp_dataset(Xva, va_meta["loan_amnt"].values, va_meta["int_rate"].values, yva)
mdp_test  = build_mdp_dataset(Xte, te_meta["loan_amnt"].values, te_meta["int_rate"].values, yte)

# Train (CPU)
algo = train_cql(mdp_train, n_steps=200_000)

# # Actions on test
# rl_actions_te = policy_actions(algo, Xte)

# # FQE (optional, robust helper)
# est_val = fqe_estimate(algo, mdp_train, mdp_valid, n_steps=100_000)
# est_te  = fqe_estimate(algo, mdp_train, mdp_test,  n_steps=100_000)
# print("FQE Estimated Policy Value — Val:", est_val, " Test:", est_te)


2025-10-29 23:30.47 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(63,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-10-29 23:30.47 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-10-29 23:30.50 [info     ] Action size has been automatically determined. action_size=2
2025-10-29 23:30.52 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(63,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-10-29 23:30.52 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-10-29 23:30.52 [info     ] Action size has been automatically determined. action_size=2
2025-10-

Epoch 1/20: 100%|██████████| 10000/10000 [03:30<00:00, 47.44it/s, loss=3.7e+3, td_loss=3.7e+3, conservative_loss=0.000979]

2025-10-29 23:34.24 [info     ] DiscreteCQL_20251029233053: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.013707966494560242, 'time_algorithm_update': 0.0071500230312347415, 'loss': 3698.7780274414063, 'td_loss': 3698.777049658203, 'conservative_loss': 0.000977902539097704, 'time_step': 0.020981254863739012} step=10000
2025-10-29 23:34.24 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251029233053/model_10000.d3



Epoch 2/20: 100%|██████████| 10000/10000 [02:43<00:00, 61.06it/s, loss=3.64e+3, td_loss=3.64e+3, conservative_loss=0]

2025-10-29 23:37.08 [info     ] DiscreteCQL_20251029233053: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.00993980803489685, 'time_algorithm_update': 0.006267837572097778, 'loss': 3644.524125854492, 'td_loss': 3644.524125854492, 'conservative_loss': 0.0, 'time_step': 0.016311634373664854} step=20000


...

2025-10-30 00:18.08 [info     ] DiscreteCQL_20251029233053: epoch=18 step=180000 epoch=18 metrics={'time_sample_batch': 0.009027961468696595, 'time_algorithm_update': 0.005775844144821167, 'loss': 3561.191135498047, 'td_loss': 3561.191135498047, 'conservative_loss': 0.0, 'time_step': 0.014903050661087037} step=180000
2025-10-30 00:18.08 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251029233053/model_180000.d3


Epoch 19/20: 100%|██████████| 10000/10000 [02:29<00:00, 66.86it/s, loss=3.56e+3, td_loss=3.56e+3, conservative_loss=1.91e-10]

2025-10-30 00:20.37 [info     ] DiscreteCQL_20251029233053: epoch=19 step=190000 epoch=19 metrics={'time_sample_batch': 0.00902596538066864, 'time_algorithm_update': 0.0057707313299179075, 'loss': 3559.7494353515626, 'td_loss': 3559.7494353515626, 'conservative_loss': 1.9073486328125e-10, 'time_step': 0.014898727560043335} step=190000
2025-10-30 00:20.37 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251029233053/model_190000.d3



Epoch 20/20: 100%|██████████| 10000/10000 [02:33<00:00, 65.25it/s, loss=3.56e+3, td_loss=3.56e+3, conservative_loss=1.77e-5]

2025-10-30 00:23.10 [info     ] DiscreteCQL_20251029233053: epoch=20 step=200000 epoch=20 metrics={'time_sample_batch': 0.009408315253257752, 'time_algorithm_update': 0.005753175973892212, 'loss': 3560.088860961914, 'td_loss': 3560.088843310547, 'conservative_loss': 1.7656707763671873e-05, 'time_step': 0.015263005924224854} step=200000
2025-10-30 00:23.10 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251029233053/model_200000.d3


In [11]:
import inspect
from src.rl_cql import _make_cql  # we already wrote this helper earlier

def train_cql_nb(mdp_train, n_steps=200_000):
    """
    Notebook override of train_cql so we ALWAYS return the model,
    even if .fit(...) returns a list.
    """
    algo = _make_cql()

    # try to build
    try:
        algo.build_with_dataset(mdp_train)
    except Exception:
        pass

    fit_sig  = inspect.signature(algo.fit)
    fit_kwds = set(fit_sig.parameters.keys())

    kwargs = {}
    if "dataset" in fit_kwds:
        kwargs["dataset"] = mdp_train
    if "n_steps" in fit_kwds:
        kwargs["n_steps"] = n_steps
    if "n_steps_per_epoch" in fit_kwds:
        kwargs["n_steps_per_epoch"] = 10_000

    # call fit
    try:
        _ = algo.fit(**kwargs)
    except TypeError:
        ds = kwargs.pop("dataset", mdp_train)
        _ = algo.fit(ds, **kwargs)

    # 🔴 IMPORTANT: always return the model, ignore what fit returned
    return algo


In [17]:
algo = train_cql_nb(mdp_train, n_steps=200_000)

2025-10-30 07:56.14 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(63,)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=2)
2025-10-30 07:56.14 [warning  ] Skip building models since they're already built.
2025-10-30 07:56.14 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20251030075614
2025-10-30 07:56.14 [info     ] Parameters                     params={'observation_shape': [63], 'action_size': 2, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 1024, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'learning_rate': 0.0003, 'optim_factory': {'type': 'adam', 'params': {'clip_grad_norm': None, 'lr_s

Epoch 1/20: 100%|██████████| 10000/10000 [04:22<00:00, 38.12it/s, loss=3.7e+3, td_loss=3.7e+3, conservative_loss=0.000884] 

2025-10-30 08:00.37 [info     ] DiscreteCQL_20251030075614: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.01772546148300171, 'time_algorithm_update': 0.008224328088760375, 'loss': 3703.496393994141, 'td_loss': 3703.495511083984, 'conservative_loss': 0.0008830552247120067, 'time_step': 0.026101881909370423} step=10000
2025-10-30 08:00.37 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075614/model_10000.d3



Epoch 2/20: 100%|██████████| 10000/10000 [04:12<00:00, 39.59it/s, loss=3.64e+3, td_loss=3.64e+3, conservative_loss=0]

2025-10-30 08:04.49 [info     ] DiscreteCQL_20251030075614: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.016973043203353883, 'time_algorithm_update': 0.008015899634361267, 'loss': 3642.636858544922, 'td_loss': 3642.636858544922, 'conservative_loss': 0.0, 'time_step': 0.025129512453079223} step=20000


...

2025-10-30 09:17.43 [info     ] DiscreteCQL_20251030075614: epoch=18 step=180000 epoch=18 metrics={'time_sample_batch': 0.020451636028289796, 'time_algorithm_update': 0.008757887268066407, 'loss': 3569.9036631103518, 'td_loss': 3569.9036631103518, 'conservative_loss': 0.0, 'time_step': 0.029380690479278566} step=180000
2025-10-30 09:17.43 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075614/model_180000.d3


Epoch 19/20: 100%|██████████| 10000/10000 [04:47<00:00, 34.82it/s, loss=3.57e+3, td_loss=3.57e+3, conservative_loss=0]

2025-10-30 09:22.30 [info     ] DiscreteCQL_20251030075614: epoch=19 step=190000 epoch=19 metrics={'time_sample_batch': 0.01874050612449646, 'time_algorithm_update': 0.009679274559020996, 'loss': 3565.9149115234377, 'td_loss': 3565.9149115234377, 'conservative_loss': 0.0, 'time_step': 0.028572401356697082} step=190000
2025-10-30 09:22.30 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075614/model_190000.d3



Epoch 20/20: 100%|██████████| 10000/10000 [04:40<00:00, 35.60it/s, loss=3.56e+3, td_loss=3.56e+3, conservative_loss=0]

2025-10-30 09:27.11 [info     ] DiscreteCQL_20251030075614: epoch=20 step=200000 epoch=20 metrics={'time_sample_batch': 0.018260151767730715, 'time_algorithm_update': 0.009531409692764283, 'loss': 3561.279971557617, 'td_loss': 3561.279971557617, 'conservative_loss': 0.0, 'time_step': 0.027950141501426697} step=200000
2025-10-30 09:27.11 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075614/model_200000.d3


In [18]:
import numpy as np

def realized_reward(actions, y_true, loan_amnt, int_rate):
    actions   = np.asarray(actions).astype(int)
    y_true    = np.asarray(y_true).astype(int)
    loan_amnt = np.asarray(loan_amnt).astype(float)
    int_rate  = np.asarray(int_rate).astype(float)

    approve_reward = np.where(y_true == 0, loan_amnt * int_rate, -loan_amnt)
    deny_reward    = np.zeros_like(approve_reward)

    return np.where(actions == 1, approve_reward, deny_reward)

def policy_value(actions, y_true, loan_amnt, int_rate):
    r = realized_reward(actions, y_true, loan_amnt, int_rate)
    return {
        "approve_rate": float((np.asarray(actions)==1).mean()),
        "mean_reward": float(r.mean()),
        "total_reward": float(r.sum())
    }

# 1) RL policy actions
rl_actions_te = algo.predict(Xte.astype(np.float32))
rl_val = policy_value(
    rl_actions_te,
    yte,
    te_meta["loan_amnt"].values,
    te_meta["int_rate"].values,
)
print("RL policy:", rl_val)

# 2) Always-approve baseline
always_approve = np.ones_like(yte)
aa_val = policy_value(
    always_approve,
    yte,
    te_meta["loan_amnt"].values,
    te_meta["int_rate"].values,
)
print("Always-approve:", aa_val)

# 3) Always-deny baseline
always_deny = np.zeros_like(yte)
ad_val = policy_value(
    always_deny,
    yte,
    te_meta["loan_amnt"].values,
    te_meta["int_rate"].values,
)
print("Always-deny:", ad_val)


RL policy: {'approve_rate': 1.0, 'mean_reward': -3316.01138286359, 'total_reward': -216011613.50249997}
Always-approve: {'approve_rate': 1.0, 'mean_reward': -3316.01138286359, 'total_reward': -216011613.50249997}
Always-deny: {'approve_rate': 0.0, 'mean_reward': 0.0, 'total_reward': 0.0}


In [15]:
from src.rl_cql import fqe_estimate, build_mdp_dataset

# we already have: mdp_train, mdp_valid, mdp_test
est_val = fqe_estimate(algo, mdp_train, mdp_valid, n_steps=10_000)
est_te  = fqe_estimate(algo, mdp_train, mdp_test,  n_steps=10_000)

print("Estimated Policy Value (FQE) — Val:", est_val)
print("Estimated Policy Value (FQE) — Test:", est_te)


2025-10-30 07:29.15 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(63,)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=2)
2025-10-30 07:29.15 [debug    ] Building models...            
2025-10-30 07:29.15 [debug    ] Models have been built.       
2025-10-30 07:29.15 [info     ] Directory is created at d3rlpy_logs/DiscreteFQE_20251030072915
2025-10-30 07:29.15 [info     ] Parameters                     params={'observation_shape': [63], 'action_size': 2, 'config': {'type': 'fqe', 'params': {'batch_size': 100, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'learning_rate': 0.0001, 'optim_factory': {'type': 'adam', 'param

Epoch 1/1: 100%|██████████| 10000/10000 [00:45<00:00, 217.63it/s, loss=3.86e+3]

2025-10-30 07:30.01 [info     ] DiscreteFQE_20251030072915: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.00188371741771698, 'time_algorithm_update': 0.0026010443449020386, 'loss': 3860.603049987793, 'time_step': 0.00456871645450592} step=10000


2025-10-30 07:30.01 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteFQE_20251030072915/model_10000.d3


AttributeError: 'DiscreteFQE' object has no attribute 'estimate_value'